In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
import warnings
warnings.filterwarnings('ignore')
from datetime import date
import time
from tqdm import tqdm

In [7]:
today = str(date.today())

In [8]:
## URL
urlTraeDetInvSS = 'https://ofi5.mef.gob.pe/inviertews/Dashboard/traeDetInvSSI'
urlVerInfObras2 = 'https://ofi5.mef.gob.pe/ssi/Ssi/verInfObras2/'
urlTraeListaEjecucionSimplePublica = 'https://ofi5.mef.gob.pe/invierte/ejecucion/traeListaEjecucionSimplePublica/'
urlExpedoenteTecnico = 'https://ofi5.mef.gob.pe/inviertews/Dashboard/verListaET/'

## Input
txt_input = '../Datos/ConsultaAvanzada_Lima_region.csv'

## OUTPUTS
txt_SSI_output = '../Datos/Resultados_SSI.txt'
txt_Infobra_output = '../Datos/Resultados_Infobra.txt'
txt_ubicacion_output = '../Datos/Resultados_ubicacion.txt'
txt_avance_output = '../Datos/Resultados_avance.txt'
txt_modificaciones_output = '../Datos/Resultados_Modificaciones.txt'
txt_ExpTecnico_output = '../Datos/Resultados_ExpTecnico.txt'
txt_detalleExpTecnico_output = '../Datos/Resultados_detalleExpTecnico.txt'

In [9]:
def call_post(SESSION, URL_CONSULTA, params):

    response = SESSION.post(URL_CONSULTA, params=params, stream=True, verify=False)
    return response

def call_get(SESSION, URL_CONSULTA):

    response = SESSION.get(URL_CONSULTA, stream=True, verify=False)
    return response

def get_traeDetInvSS(SESSION, codigoIdentificador, txt_SSI_output, today, tipo='SIAF'):
    
    parametros = {
        "id": codigoIdentificador,
        "tipo": tipo
    }
    
    resultadosTraeDetInvSS = call_post(SESSION, urlTraeDetInvSS, parametros)
    
    if resultadosTraeDetInvSS.status_code == 429:
        time.sleep(int(resultadosTraeDetInvSS.headers["Retry-After"]))
        resultadosTraeDetInvSS = call_post(SESSION, urlTraeDetInvSS, parametros)  
    
    if not resultadosTraeDetInvSS.ok:
        return None
    
    pdTraeDetInvSS = pd.DataFrame(data=resultadosTraeDetInvSS.json(), dtype='str')
    pdTraeDetInvSS['FECHA_ACTUALIZACION'] = today
    lineas = pdTraeDetInvSS.fillna('').agg('|'.join, axis=1).values
    
    for linea in lineas:
        file_output = open(txt_SSI_output, 'a', encoding='utf-8')
        file_output.write(linea + '\n')
        file_output.close()
        
def get_infobra(SESSION, codigoSNIP, txt_Infobra_output, txt_ubicacion_output, txt_avance_output, today):
    
    resultadosVerInfObras2 = call_get(SESSION, urlVerInfObras2+str(codigoSNIP))
    
    if resultadosVerInfObras2.status_code == 429:
        time.sleep(int(resultadosVerInfObras2.headers["Retry-After"]))
        resultadosVerInfObras2 = call_get(SESSION, urlVerInfObras2+str(codigoSNIP))
    
    if not resultadosVerInfObras2.ok:
        return None
    
    if (resultadosVerInfObras2.text == '') | (resultadosVerInfObras2.text == '[]'):
        return None
    
    resultadosVerInfObras2Json = resultadosVerInfObras2.json()
    
    pdresultadosVerInfObras2 = pd.DataFrame(data=resultadosVerInfObras2Json, dtype='str').drop(['obj_SNO_OBRA_SNO_COORDENADAS', 'obj_SNO_OBRA_UBIGEO', 'obj_SNO_AVANCE', 'ExtensionData'], axis=1)
    pdresultadosVerInfObras2['CODIGO_UNICO'] = str(codigoIdentificador)
    pdresultadosVerInfObras2['FECHA_ACTUALIZACION'] = today
    
    lineas = pdresultadosVerInfObras2.fillna('').agg('|'.join, axis=1).values
    
    for linea in lineas:
        file_output = open(txt_Infobra_output, 'a', encoding='utf-8')
        file_output.write(linea + '\n')
        file_output.close()
        
    pdObrasCoordenadas = pd.DataFrame([resultadosVerInfObras2Json[0]['obj_SNO_OBRA_SNO_COORDENADAS']]).drop(['ExtensionData'], axis=1)
    pdObrasCoordenadas['CODIGO_UNICO'] = codigoIdentificador
    
    pdObrasUbigeo = pd.DataFrame([resultadosVerInfObras2Json[0]['obj_SNO_OBRA_UBIGEO']]).drop(['ExtensionData'], axis=1)
    pdObrasUbigeo['CODIGO_UNICO'] = codigoIdentificador
    
    pdUbicacion = pd.merge(pdObrasCoordenadas, pdObrasUbigeo, how='left', on='CODIGO_UNICO')
    pdUbicacion['FECHA_ACTUALIZACION'] = today
    
    lineas = pdUbicacion.fillna('').astype('str').agg('|'.join, axis=1).values
    
    for linea in lineas:
        file_output = open(txt_ubicacion_output, 'a', encoding='utf-8')
        file_output.write(linea + '\n')
        file_output.close()
        
    if resultadosVerInfObras2Json[0]['obj_SNO_AVANCE'] is not None:
    
        pdObrasAvance = pd.DataFrame(resultadosVerInfObras2Json[0]['obj_SNO_AVANCE']).drop(['ExtensionData'], axis=1)
        pdObrasAvance['CODIGO_UNICO'] = codigoIdentificador
        pdObrasAvance['FECHA_ACTUALIZACION'] = today

        lineas = pdObrasAvance.fillna('').astype('str').agg('|'.join, axis=1).values

        for linea in lineas:
            file_output = open(txt_avance_output, 'a', encoding='utf-8')
            file_output.write(linea + '\n')
            file_output.close()
        
def get_ejecucionSimple(SESSION, codigoIdentificador, txt_modificaciones_output, today):
    
    resultadosListaEjecucionSimple = call_get(SESSION, urlTraeListaEjecucionSimplePublica+str(codigoIdentificador))
    
    if resultadosListaEjecucionSimple.status_code == 429:
        time.sleep(int(resultadosListaEjecucionSimple.headers["Retry-After"]))
        resultadosListaEjecucionSimple = call_get(SESSION, urlTraeListaEjecucionSimplePublica+str(codigoIdentificador))
    
    if not resultadosListaEjecucionSimple.ok:
        return None
    
    htmlListaEjecucionSimple = BeautifulSoup(resultadosListaEjecucionSimple.content, 'html.parser')
    pdModificaciones = pd.read_html(resultadosListaEjecucionSimple.text)[0]
    
    pdModificaciones['Monto actualizado S/'] = pdModificaciones['Monto actualizado S/'].apply(lambda x: str(x).replace('S/. ', '').replace(',', '').strip())
    pdModificaciones['CODIGO_UNICO'] = codigoIdentificador
    pdModificaciones['FECHA_ACTUALIZACION'] = today
    
    lineas = pdModificaciones.fillna('').astype('str').agg('|'.join, axis=1).values
    
    for linea in lineas:
        file_output = open(txt_modificaciones_output, 'a', encoding='utf-8')
        file_output.write(linea + '\n')
        file_output.close()
        
def get_expTecnico(SESSION, codigoIdentificador, txt_ExpTecnico_output, txt_detalleExpTecnico_output, today):
    
    resultadosExpedoenteTecnico = call_get(SESSION, urlExpedoenteTecnico+str(codigoIdentificador))
    
    if resultadosExpedoenteTecnico.status_code == 429:
        time.sleep(int(resultadosExpedoenteTecnico.headers["Retry-After"]))
        resultadosExpedoenteTecnico = call_get(SESSION, urlExpedoenteTecnico+str(codigoIdentificador))
        
    if not resultadosExpedoenteTecnico.ok:
        return None
    
    if (resultadosExpedoenteTecnico.text == '') | (resultadosExpedoenteTecnico.text == '[]'):
        return None
    
    resultadosExpedoenteTecnicoJson = resultadosExpedoenteTecnico.json()
    
    pdExpedienteTecnico = pd.DataFrame(resultadosExpedoenteTecnicoJson)
    pdExpedienteTecnico['FECHA_ACTUALIZACION'] = today
    
    lineas = pdExpedienteTecnico.fillna('').astype('str').agg('|'.join, axis=1).values
    for linea in lineas:
        file_output = open(txt_ExpTecnico_output, 'a', encoding='utf-8')
        file_output.write(linea + '\n')
        file_output.close()
        
    ID_EXP_TECNICO = resultadosExpedoenteTecnicoJson[0]['ID_EXP_TECNICO']
    
    urlExpedoenteTecnicoDetalle = 'https://ofi5.mef.gob.pe/inviertews/Dashboard/verDetalleET?id='+str(ID_EXP_TECNICO)+'&tipo=NVO'
    resultadosExpedoenteTecnicoDetalle = call_get(SESSION, urlExpedoenteTecnicoDetalle)
    
    if resultadosExpedoenteTecnicoDetalle.status_code == 429:
        time.sleep(int(resultadosExpedoenteTecnicoDetalle.headers["Retry-After"]))
        resultadosExpedoenteTecnicoDetalle = call_get(SESSION, urlExpedoenteTecnicoDetalle)
    
    if not resultadosExpedoenteTecnicoDetalle.ok:
        return None
    
    detalleExpedienteTecnico = pd.DataFrame(resultadosExpedoenteTecnicoDetalle.json())
    detalleExpedienteTecnico['FECHA_ACTUALIZACION'] = today
    lineas = detalleExpedienteTecnico.fillna('').astype('str').agg('|'.join, axis=1).values
    
    for linea in lineas:
        file_output = open(txt_detalleExpTecnico_output, 'a', encoding='utf-8')
        file_output.write(linea + '\n')
        file_output.close()

In [10]:
def get_contratos(SESSION, codigoContrato, txt_contrato_output, txt_contratoCUI_output, txt_contratoGarantia_output, today):
    
    resultadosVerContrato = call_get(SESSION, urlVerContrato+str(codigoContrato))
    
    if resultadosVerContrato.status_code == 429:
        time.sleep(int(resultadosVerContrato.headers["Retry-After"]))
        resultadosVerContrato = call_get(SESSION, urlVerContrato+str(codigoContrato))
    
    if not resultadosVerContrato.ok:
        return None
    
    if (resultadosVerContrato.text == '') | (resultadosVerContrato.text == '[]'):
        return None
    
    dfResultadosVerContrato = pd.DataFrame([resultadosVerContrato.json()])
    dfResultadosVerContrato['FECHA_ACTUALIZACION'] = today
    
    for columna in dfResultadosVerContrato.columns:
        dfResultadosVerContrato[columna] = dfResultadosVerContrato[columna].apply(lambda x: str(x).replace('|', ' ').replace("\n", " "))
    
    lineas = dfResultadosVerContrato.fillna('').astype('str').agg('|'.join, axis=1).values
    
    for linea in lineas:
        file_output = open(txt_contrato_output, 'a', encoding='utf-8')
        file_output.write(linea + '\n')
        file_output.close()
        
    if (resultadosVerContrato.json()['listaProyectoContrato'] is not None)&(resultadosVerContrato.json()['listaProyectoContrato'] != '[]'):
    
        pdContratoCUI = pd.DataFrame(resultadosVerContrato.json()['listaProyectoContrato'])
        pdContratoCUI['idContrato'] = codigoContrato
        pdContratoCUI['FECHA_ACTUALIZACION'] = today

        for columna in pdContratoCUI.columns:
            pdContratoCUI[columna] = pdContratoCUI[columna].apply(lambda x: str(x).replace('|', ' ').replace("\n", " "))
        
        lineas = pdContratoCUI.fillna('').astype('str').agg('|'.join, axis=1).values

        for linea in lineas:
            file_output = open(txt_contratoCUI_output, 'a', encoding='utf-8')
            file_output.write(linea + '\n')
            file_output.close()
            
    if (resultadosVerContrato.json()['listaGarantiaContrato'] is not None)&(resultadosVerContrato.json()['listaGarantiaContrato'] != '[]'):
    
        pdGarantia = pd.DataFrame(resultadosVerContrato.json()['listaGarantiaContrato'])
        pdGarantia['idContrato'] = codigoContrato
        pdGarantia['FECHA_ACTUALIZACION'] = today

        for columna in pdGarantia.columns:
            pdGarantia[columna] = pdGarantia[columna].apply(lambda x: str(x).replace('|', ' ').replace("\n", " "))
        
        lineas = pdGarantia.fillna('').astype('str').agg('|'.join, axis=1).values

        for linea in lineas:
            file_output = open(txt_contratoGarantia_output, 'a', encoding='utf-8')
            file_output.write(linea + '\n')
            file_output.close()

In [8]:
SESSION = requests.Session()

In [13]:
codigos_revisados = []

In [14]:
SESSION = requests.Session()

In [15]:
#if __name__ == '__main__':
    
with open(txt_input,'r') as f:
    content = f.readlines()
    for i, line in tqdm(enumerate(content)):
        codigoIdentificador = line.split(',')[0].strip()
        codigoSNIP = line.split(',')[1].strip()
        
        if (i==0) | (codigoIdentificador in codigos_revisados) :
        #if (i < 12593) | (codigoIdentificador in codigos_revisados) :
            continue
        
        if (codigoIdentificador==''):
            get_traeDetInvSS(SESSION, codigoIdentificador, txt_SSI_output, today, tipo = 'SNIP')
            get_infobra(SESSION, codigoSNIP, txt_Infobra_output, txt_ubicacion_output, txt_avance_output, today)
            
        else:

            get_traeDetInvSS(SESSION, codigoIdentificador, txt_SSI_output, today)
            get_infobra(SESSION, codigoSNIP, txt_Infobra_output, txt_ubicacion_output, txt_avance_output, today)
            get_ejecucionSimple(SESSION, codigoIdentificador, txt_modificaciones_output, today)
            get_expTecnico(SESSION, codigoIdentificador, txt_ExpTecnico_output, txt_detalleExpTecnico_output, today)
        
        #print('Completo:', codigoIdentificador, codigoSNIP)
        
        codigos_revisados.append(codigoIdentificador)
        

1877it [19:06,  1.64it/s]


### Contratos

In [7]:
urlVerContrato = 'http://contratos.seace.gob.pe:9000/api/bus/contrato/idContrato/'
#txt_contrato_output = '../Datos/Resultados_Contratos.txt'
txt_contrato_output = '../Datos/Resultados_Contratos_d2.txt'
txt_contratoCUI_output = '../Datos/Resultados_ContratosCUI.txt'
txt_contratoGarantia_output = '../Datos/Resultados_ContratosGarantia.txt'

today = str(date.today())
SESSION = requests.Session()

In [8]:
contratos = pd.read_csv('../Datos/Listado_Contratos.csv', sep='|')
contratos.sort_values(by='FECHA_PUBLICACION_CONTRATO', ascending=False, inplace=True)

In [9]:
contratos.head()

,CODIGOCONVOCATORIA,N_COD_CONTRATO,DESCRIPCION_PROCESO,FECHA_PUBLICACION_CONTRATO,FECHA_SUSCRIPCION_CONTRATO,FECHA_VIGENCIA_INICIAL,FECHA_VIGENCIA_FINAL,FECHA_VIGENCIA_FIN_ACTUALIZADA,CODIGO_CONTRATO,NUM_CONTRATO,MONTO_CONTRATADO_TOTAL,MONTO_CONTRATADO_ITEM,MONTO_ADICIONAL,MONTO_REDUCCION,MONTO_PRORROGA,MONTO_COMPLEMENTARIO,URLCONTRATO,CODIGOENTIDAD,NUM_ITEM,MONEDA,RUC_CONTRATISTA,RUC_DESTINATARIO_PAGO,TIENERESOLUCION,CUI,SNIP
252696,756015,2095549,SUPERVISION DE LA OBRA: MEJORAMIENTO DEL SERVI...,2022-04-09 19:48:32,2021-12-30 00:00:00,2022-01-10 00:00:00,2023-02-04 00:00:00,2023-02-04 00:00:00,2095549,CONTRATO CONCURSO PUBLICO N° 001-2021-MDLC/ABAST,488850.00,488850.00,NaN,NaN,NaN,NaN,http://contratos.seace.gob.pe:9045/api/documen...,703,1.0,Soles,484939,20608853651,NO,NaN,NaN
264841,781607,2108677,ADQUISICION REPUESTOS PARA ESTACIONES AUTOMATI...,2022-04-09 19:12:28,2022-04-06 00:00:00,2022-04-07 00:00:00,2022-07-20 00:00:00,2022-07-20 00:00:00,2108677,003-2022,97790.00,97790.00,NaN,NaN,NaN,NaN,http://contratos.seace.gob.pe:9045/api/documen...,10248,1.0,Soles,20509585661,20509585661,NO,NaN,NaN
264534,780572,2108676,CONTRATACION DE SUMINISTRO DE COMBUSTIBLE (DIE...,2022-04-09 12:22:11,2022-03-23 00:00:00,2022-03-24 00:00:00,2022-09-30 00:00:00,2022-09-30 00:00:00,2108676,CONTRATO N° 065-2022-MPM,214700.00,214700.00,NaN,NaN,NaN,NaN,http://contratos.seace.gob.pe:9045/api/documen...,995,1.0,Soles,20560151757,20560151757,NO,NaN,NaN
266592,792181,2108675,Suministro de alimento balanceado (hipoalergén...,2022-04-09 12:18:42,2022-04-09 00:00:00,2022-04-11 00:00:00,2022-04-15 00:00:00,2022-04-15 00:00:00,2108675,02-2022-VII DIRTEPOL LIMA,53406.72,53406.72,NaN,NaN,NaN,NaN,http://contratos.seace.gob.pe:9045/api/documen...,10150,1.0,Soles,20513345748,20513345748,NO,NaN,NaN
266483,790046,2108674,Servicio de arrendamiento de un inmueble para ...,2022-04-09 10:27:13,2022-04-08 00:00:00,2022-04-09 00:00:00,2025-06-07 00:00:00,2025-06-07 00:00:00,2108674,Contrato N° 012-2022-s-p-csjli-pj,10622808.36,10622808.36,NaN,NaN,NaN,NaN,http://contratos.seace.gob.pe:9045/api/documen...,200428,1.0,Soles,20514441163,20514441163,NO,NaN,NaN


In [10]:
listaContratos = list(contratos['N_COD_CONTRATO'].unique().tolist())

In [19]:
listaContratos_f = listaContratos[::-1].copy()

In [27]:
codigos_revisados= []

In [28]:
for codigoContrato in tqdm(listaContratos_f[117190:]):
    get_contratos(SESSION, str(codigoContrato), txt_contrato_output, txt_contratoCUI_output, txt_contratoGarantia_output, today)
    
    codigos_revisados.append(codigoContrato)

100%|██████████████████████████████████████████████████████████████████████████| 19031/19031 [1:27:13<00:00,  3.64it/s]
